## Fetching weather data from openweathermap.org

It's possible to get data for the past 5 days with an unpaid account, but getting data any further back costs money.
As of now I only have data more than 5 days back for one station: Axis-Brightwood. I'm considering buying historical data for that one site, but for now the following code will fetch recent data for all sites at which I'm actively collecting data.
I need to manually run this notebook once a day until I get around to copy-pasting this into a script and automating that task.

In [1]:
import json, os, requests, time
from datetime import datetime, timedelta
import pymysql
import sqlalchemy as SQL
from urllib.parse import quote_plus as QP
HOME = os.path.expanduser('~')
WEATHER_DIR = os.path.join(HOME, 'Data', 'Storage', 'AlertWF', 'weather')

if not os.path.exists(WEATHER_DIR):
    os.makedirs(WEATHER_DIR)
assert os.path.isdir(WEATHER_DIR)

secrets_path = os.path.join(HOME, 'Documents', 'secrets.json')
with open(secrets_path,'r') as secrets_file:
    secrets = json.load(secrets_file)
    API_KEY = secrets['openweathermap.org']['api_key']
    _sql_secrets = secrets['mysql']
    sql_passwd = _sql_secrets['PASSWD']
    sql_usr = _sql_secrets['USER']
    sql_host = _sql_secrets['HOST']
    sql_port = _sql_secrets['PORT']
    sql_db = _sql_secrets['DB']

WEATHER_API = 'https://api.openweathermap.org/data/2.5/onecall/timemachine'
SQL_URL = f"mysql+pymysql://{sql_usr}:{QP(sql_passwd)}@{sql_host}:{sql_port}/{sql_db}"
SQL_ENGINE = SQL.create_engine(SQL_URL)
%load_ext sql
%sql $SQL_URL

In [2]:
def get_hourly_weather(lon:float, lat:float, date:datetime) -> dict:
    dt = int(date.timestamp())
    prm = dict(lon=lon,lat=lat,dt=dt,units='metric',appid=API_KEY)
    result = requests.get(WEATHER_API, params=prm)
    assert result.status_code == 200, \
        f'status code: {result.status_code}\nURL: {result.url}'
    time.sleep(1) ## Rate-limited API
    return result.json()

def days_ago(n: int) -> datetime:
    d = datetime.date(datetime.today())-timedelta(days=n)
    dt = datetime.fromordinal(d.toordinal())
    return dt

In [3]:
query_one = 'SELECT lon, lat FROM stations WHERE id=:station'
query_all = 'SELECT DISTINCT station FROM stations st INNER JOIN images im ON st.id=im.station;'

with SQL_ENGINE.connect().execution_options(autocommit=True) as conn:
    result = conn.execute(query_all)
    stations = [row[0] for row in result.fetchall()]
    for station in stations:
        print('Processing station:',station)
        query = SQL.text(query_one).bindparams(SQL.bindparam("station", type_=SQL.String))
        lon, lat = conn.execute(query, dict(station=station)).fetchone()
        
        #for n in range(5):
        for n in [1]:
            dt = days_ago(n)
            weather = get_hourly_weather(lon, lat, dt)
            dstamp = dt.strftime('%Y-%m-%d')
            fname = f'{station}-{dstamp}.json'
            path = os.path.join(WEATHER_DIR, fname)
            with open(path, 'w') as data_file:
                json.dump(weather, data_file)

print('Done!')        

Processing station: Axis-Alpine
Processing station: Axis-Brightwood
Processing station: Axis-DeerHorn2
Processing station: Axis-FayRanch2
Processing station: Axis-GrizzlyPeakLookout2
Processing station: Axis-HolidayLake2
Processing station: Axis-Indiana
Processing station: Axis-Steens
Processing station: Axis-Strawberry2
Processing station: Axis-WestMtnUtah
Processing station: Axis-WhitneyPortal2
Done!
